In [ ]:
from nltk.collocations import *
import nltk
import pandas as pd
import numpy as np
import re
import nltk
import nltk, re, string
from nltk.corpus import stopwords
import nltk
import math
import string
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem.porter import *
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem.porter import*
import os
import sys
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import math
import operator



# Read created dictionary
pattern=r'\w[\w\',-]*\w'
with open("/Users/shenzongqi/Desktop/PythonProject/Stevens/2022_Research/envir_words.txt",'r') as f:
    envir = [line.strip() for line in f]
envir = " ".join(envir)
envir=nltk.regexp_tokenize(envir.lower(), pattern)

### Read company 10k report

def clean_report(text):
#tokens=nltk.regexp_tokenize(text.lower(), pattern)
    stop_words = stopwords.words('english')
### Clean 10k Report data: (Lower, remove stop words)
    tokens = [token.strip() \
              for token in nltk.word_tokenize(text.lower()) \
              if token.strip() not in stop_words and \
              token.strip() not in string.punctuation]
    return tokens


# Tf_Idf matrix

# Function1
def loadDataSet(tokens):
    dataset = tokens
    classVec = [0, 1, 0, 1, 0, 1]  # class
    return dataset, classVec

# Function2
def feature_select(dataset):
    # Total word frequency statistics
    doc_frequency = defaultdict(int) #Record the number of occurrences of each word, which can be understood as a variable-length list, as long as you index it, it will automatically expand the column
    for file in dataset:
        for word in file:
            doc_frequency[word] += 1
    # Calculate the TF value of each word
    word_tf = {}  # Store the tf value of each word
    for i in doc_frequency:
        word_tf[i] = doc_frequency[i] / sum(doc_frequency.values()) #sum(doc.frequency.values)

    # Calculate the IDF value of each word
    doc_num = len(dataset)
    word_idf = {}  # Store the idf value of each word
    word_doc = defaultdict(int)  # Stores the number of documents that contain the word
    for word in doc_frequency:
        for file in dataset:
            if word in file:
                word_doc[word] += 1
    #
    for word in doc_frequency:
        word_idf[word] = math.log(doc_num / (word_doc[word] + 1))

    # Calculate the value of TF*IDF for each word
    word_tf_idf = {}
    for word in doc_frequency:
        word_tf_idf[word] = word_tf[word] * word_idf[word]

    # Sort dictionary by value from largest to smallest
    dict_feature_select = sorted(word_tf_idf.items(), key=operator.itemgetter(1), reverse=True)
    return dict_feature_select


#####Test
def Analysis_10k(text):
    tokens = clean_report(text)
    data_list, label_list = loadDataSet(tokens)
    features = feature_select(data_list)  # TF-IDF values for all words
    df = pd.DataFrame(features)
    Fin = sum(df[1] / len(df[1]))
    return Fin





# Loop all files in dictionary
import os
import codecs
path = "/Users/shenzongqi/Desktop/PythonProject/Stevens/2022_Research/Data/inputmda_01" #folder directory
files= os.listdir(path) #Get the names of all files in a folder
scores = []
for file in files: #Traverse folders
    position = path+'//'+ file #Construct an absolute path, "\\", one of '\' is an escape character
    print (position)
    with codecs.open(position, 'r', encoding='utf-8',
                     errors='ignore') as f:
        text = [line.strip() for line in f]
        text = text[0]
        score = Analysis_10k(text)
        score = (file[:-4],score)
    scores.append(score)
scores = pd.DataFrame(scores)
print (scores)
scores.columns = ["CIK","Score"]


scores.to_excel("/Users/shenzongqi/Desktop/PythonProject/Stevens/2022_Research/Data/scores.xls", index=False)


In [ ]:
import pandas as pd
scores_read = pd.read_excel("/Users/shenzongqi/Desktop/PythonProject/Stevens/2022_Research/Data/scores.xls")

def add_Year(df):
    # Creat column Year with 0
    df['Year'] = 0
    for i in range(0, 46887):
        s = df.iloc[i, 0]
        df.iloc[i, 0] = s[:-4]
        df.iloc[i, 2] = s.split('-')[1]
    # add 20; change column names
    for i in range(0, 46887):
        df.iloc[i, 2] = "20" + df.iloc[i, 2]

    df["CIK"] = 0
    for i in range(0, 46887):
        s = df.iloc[i, 0]
        df.iloc[i, 3] = s.split('_')[0]
        df.iloc[i,0] = s.split('_')[1]
    return df

scores = add_Year(scores_read)
#scores.to_excel("/Users/shenzongqi/Desktop/PythonProject/Stevens/2022_Research/Data/scores_Year.xls", index=False)
scores = pd.read_excel("/Users/shenzongqi/Desktop/PythonProject/Stevens/2022_Research/Data/scores_Year.xls")



Co_name = pd.read_csv("/Users/shenzongqi/Desktop/PythonProject/Stevens/2022_Research/Data/CIK-Co_name1993-2021.csv")
Co_name.info()
Co_name = Co_name[["CIK","ACC_NUM","FORM_TYPE","CoName"]]
# names = Co_name.loc[Co_name[]]

# change CIK to int type
scores['CIK']=scores['CIK'].astype('int64')
Co_name['CIK']=Co_name['CIK'].astype('int64')

# merge
df2 = pd.merge(scores, Co_name, on='ACC_NUM')
fin_data = df2.loc[df2.iloc[:,5] == '10-K',:]
fin_data.info()
fin_data_2 = fin_data.drop(labels='CIK_x',axis=1)
fin_data_2.drop_duplicates()
fin_data_2.rename(columns = {"CIK_y":"CIK"}, inplace=True)
#fin_data_2.to_excel("/Users/shenzongqi/Desktop/PythonProject/Stevens/2022_Research/Data/Score_Coname.xls", index=False)

#_________________________________________________________________________________________________________________________
fin_data_2 = pd.read_excel("/Users/shenzongqi/Desktop/PythonProject/Stevens/2022_Research/Data/Score_Coname.xls")



# merge data with ticker

ticker = pd.read_excel("/Users/shenzongqi/Desktop/PythonProject/Stevens/2022_Research/Data/ticker.xlsx")
newDF_withticker = pd.merge(fin_data_2, ticker, on='CIK')
newDF_withticker.drop(["Unnamed: 0","Unnamed: 0.1"],axis=1, inplace = True)
#newDF_withticker.to_excel("/Users/shenzongqi/Desktop/PythonProject/Stevens/2022_Research/Data/Year_Scores/newDF_withticker.xlsx")
## newDF_withticker: ACC_NUM + Score + Year + CIK + FORM_TYPE + CoName + Tikcer




##### start from here
import pandas as pd
newDF_withticker = pd.read_excel("/Users/shenzongqi/Desktop/PythonProject/Stevens/2022_Research/Data/Year_Scores/newDF_withticker.xlsx")

##################################################################################################
sc_2017 = newDF_withticker.loc[newDF_withticker.iloc[:,3] == 2017,:]
sc_2017.drop_duplicates(subset=['CoName'],inplace = True)
sc_2017.drop_duplicates(subset=['Ticker'],inplace = True)
# check unique
points = sc_2017.CoName.unique()
sc_2017.to_excel("/Users/shenzongqi/Desktop/PythonProject/Stevens/2022_Research/Data/Year_Scores/sc_2017.xlsx")

##################################################################################################
sc_2018 = newDF_withticker.loc[newDF_withticker.iloc[:,3] == 2018 ,:]
sc_2018.drop_duplicates(subset=['CoName'],inplace = True)
sc_2018.drop_duplicates(subset=['Ticker'],inplace = True)
# check unique
points = sc_2018.CoName.unique()
sc_2018.to_excel("/Users/shenzongqi/Desktop/PythonProject/Stevens/2022_Research/Data/Year_Scores/sc_2018.xlsx")

##################################################################################################
sc_2019 = newDF_withticker.loc[newDF_withticker.iloc[:,3] == 2019,:]
sc_2019.drop_duplicates(subset=['CoName'],inplace = True)
sc_2019.drop_duplicates(subset=['Ticker'],inplace = True)
# check unique
points = sc_2019.CoName.unique()
sc_2019.to_excel("/Users/shenzongqi/Desktop/PythonProject/Stevens/2022_Research/Data/Year_Scores/sc_2019.xlsx")

##################################################################################################
sc_2020 = newDF_withticker.loc[newDF_withticker.iloc[:,3] == 2020 ,:]
sc_2020.drop_duplicates(subset=['CoName'],inplace = True)
sc_2020.drop_duplicates(subset=['Ticker'],inplace = True)
# check unique
points = sc_2020.CoName.unique()
sc_2020.to_excel("/Users/shenzongqi/Desktop/PythonProject/Stevens/2022_Research/Data/Year_Scores/sc_2020.xlsx")

##################################################################################################
sc_2021 = newDF_withticker.loc[newDF_withticker.iloc[:,3] == 2021,:]
sc_2021.drop_duplicates(subset=['CoName'],inplace = True)
sc_2021.drop_duplicates(subset=['Ticker'],inplace = True)
# check unique
points = sc_2021.CoName.unique()
sc_2021.to_excel("/Users/shenzongqi/Desktop/PythonProject/Stevens/2022_Research/Data/Year_Scores/sc_2021.xlsx")






###################
# reload file
import pandas as pd
### Read xlsx file for each year
sc_2017 = pd.read_excel("/Users/shenzongqi/Desktop/PythonProject/Stevens/2022_Research/Data/Year_Scores/sc_2017.xlsx")
sc_2017["Judge"] = 0
sc_2017.loc[sc_2017["Score"] > 0.04, "Judge"] = "Yes"
sc_2017.loc[sc_2017["Ticker"]=="", "Score"]


sc_2018 = pd.read_excel("/Users/shenzongqi/Desktop/PythonProject/Stevens/2022_Research/Data/Year_Scores/sc_2018.xlsx")
sc_2018["Judge"] = 0
sc_2018.loc[sc_2018["Score"] > 0.04, "Judge"] = "Yes"
sc_2018.loc[sc_2018["Ticker"]=="aapl", "Score"]

sc_2019 = pd.read_excel("/Users/shenzongqi/Desktop/PythonProject/Stevens/2022_Research/Data/Year_Scores/sc_2019.xlsx")
sc_2019["Judge"] = 0
sc_2019.loc[sc_2019["Score"] > 0.04, "Judge"] = "Yes"
sc_2019.loc[sc_2019["Ticker"]=="aapl", "Score"]

sc_2020 = pd.read_excel("/Users/shenzongqi/Desktop/PythonProject/Stevens/2022_Research/Data/Year_Scores/sc_2020.xlsx")
sc_2020["Judge"] = 0
sc_2020.loc[sc_2020["Score"] > 0.04, "Judge"] = "Yes"
sc_2020.loc[sc_2020["Ticker"]=="aapl", "Score"]

sc_2021 = pd.read_excel("/Users/shenzongqi/Desktop/PythonProject/Stevens/2022_Research/Data/Year_Scores/sc_2021.xlsx")
sc_2021["Judge"] = 0
sc_2021.loc[sc_2021["Score"] > 0.04, "Judge"] = "Yes"
sc_2021.loc[sc_2021["Ticker"]=="aapl", "Score"]





import numpy as np
sc_non = newDF_withticker.loc[newDF_withticker.iloc[:,2] != 0, "Score"]
np.mean(sc_non)
